In [30]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [31]:
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential  #函數學習機
from tensorflow.keras.layers import Conv2D, MaxPooling2D  #決定大小
from tensorflow.keras.layers import Dense, Flatten  #拉平，才能進入fully connected
from tensorflow.keras.optimizers import SGD

### 1. 讀入 MNSIT 數據集

In [32]:
from tensorflow.keras.datasets import mnist

In [33]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

### 2. 資料整理

### Channel

CNN 要注意一張圖有多少個 channel(例如RGB三源色就有三個), 開始我們因為只有灰階, 所以只有一個 channel。因此我們要轉一下我們的資料格式:

    (28,28) --> (28, 28, 1)

In [34]:
x_train = x_train.reshape(60000, 28, 28, 1) / 255

In [35]:
x_test = x_test.reshape(10000, 28, 28, 1) / 255

In [36]:
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

### 3. step 1: 打造函數學習機 (CNN)

In [37]:
model = Sequential()  #打開空白的函數學習機

In [38]:
model.add(Conv2D(14, (5,5), padding='same',          #有8個filter，設定3*3的大小(第二次改成14個filter，5*5的大小)
                input_shape=(28,28,1),
                activation='relu'))

In [39]:
model.add(MaxPooling2D(pool_size=(2,2)))  #切成2*2，從中擷取最大數字

In [40]:
model.add(Conv2D(28, (5,5), padding='same',         #filter個數要前一層的2倍或以上 #第一次已有input_shape
                activation='relu'))

In [41]:
model.add(MaxPooling2D(pool_size=(2,2)))

In [42]:
model.add(Conv2D(58, (5,5), padding='same',
                activation='relu'))

In [43]:
model.add(MaxPooling2D(pool_size=(2,2)))

In [44]:
model.add(Flatten())

In [46]:
model.add(Dense(92, activation='relu')) #全部連接的神經元個數可自己決定，這次設定92個

In [47]:
model.add(Dense(10, activation='softmax'))   #固定輸出10個

#### 看一下我們的神經網路

In [48]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 28, 28, 14)        364       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 14, 14, 14)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 14, 14, 28)        9828      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 7, 7, 28)          0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 7, 7, 58)          40658     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 3, 3, 58)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 522)              

第一行：5乘5=25個變數，+1個偏值，總共26個，又有14個filter，所以總共有26乘14=364個filter要微調


#### 組裝

In [49]:
model.compile(loss='mse', optimizer=SGD(lr=0.087),
             metrics=['accuracy'])

### 4. step 2: fit

In [50]:
model.fit(x_train, y_train, batch_size=128, epochs=12)

Epoch 1/12
469/469 [==============================] - 2s 4ms/step - loss: 0.0899 - accuracy: 0.1208
Epoch 2/12
469/469 [==============================] - 2s 4ms/step - loss: 0.0898 - accuracy: 0.3114
Epoch 3/12
469/469 [==============================] - 2s 4ms/step - loss: 0.0895 - accuracy: 0.3802
Epoch 4/12
469/469 [==============================] - 2s 4ms/step - loss: 0.0888 - accuracy: 0.3896
Epoch 5/12
469/469 [==============================] - 2s 4ms/step - loss: 0.0860 - accuracy: 0.3728
Epoch 6/12
469/469 [==============================] - 2s 4ms/step - loss: 0.0587 - accuracy: 0.5887
Epoch 7/12
469/469 [==============================] - 2s 4ms/step - loss: 0.0275 - accuracy: 0.8161
Epoch 8/12
469/469 [==============================] - 2s 4ms/step - loss: 0.0178 - accuracy: 0.8844
Epoch 9/12
469/469 [==============================] - 2s 4ms/step - loss: 0.0133 - accuracy: 0.9143
Epoch 10/12
469/469 [==============================] - 2s 4ms/step - loss: 0.0107 - accuracy: 0.9313

### Step 3. 預測

In [51]:
result = model.predict_classes(x_test)

### 看看測試資料表現如何

In [52]:
loss, acc = model.evaluate(x_test, y_test)

313/313 [==============================] - 1s 2ms/step - loss: 0.0072 - accuracy: 0.9544


In [53]:
print(f'測試資料的正確率為 {acc*100:.2f}%')

測試資料的正確率為 95.44%


In [54]:
def my_predict(n):
    print('我可愛的 CNN 預測是', result[n])
    X = x_test[n].reshape(28,28)
    plt.imshow(X, cmap='Greys')

In [55]:
from ipywidgets import interact_manual

In [56]:
interact_manual(my_predict, n=(0, 9999));

interactive(children=(IntSlider(value=4999, description='n', max=9999), Button(description='Run Interact', sty…

### 把我們的 model 存起來

In [57]:
model.save('myCNNmodel.h5')